In [3]:
import numpy as np
import numba
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, HoverTool, CustomJS
output_notebook()


Loading BokehJS ...

In [96]:
import mech,electric
x = np.zeros((1,1))+10.0
obj_w = mech.wind_turbine()
obj_w.x = x
obj_w.f_eval()
obj_pr = electric.pmsm_rectifier()
obj_pr.v_dc_src

0.0

In [288]:
@numba.jit(nopython=True)
def run_eval(nu=8.0, v_dc_src=50.0):
    N_steps = int(0.4e6)
    N_states = 6
    x = np.zeros((N_states,1))+15
    f = np.zeros((N_states,1))
    X = np.zeros((N_states,N_steps))
    out = np.zeros((3,N_steps))
    #self.out_cplx = np.zeros((72,N_steps),np.complex128)
        
    T = np.zeros((N_steps,1))
    
    obj_w = mech.wind_turbine()
    obj_pr = electric.pmsm_rectifier()
    
    obj_w.nu = nu
    obj_pr.v_dc_src = v_dc_src
    obj_pr.L=0.001
    obj_pr.R=0.01
    obj_pr.R_dc=0.01
    obj_pr.Phi =  3.0
    x[4] =100.0
    dt = 10.0e-6
    t = 0.0
    X[:,0] = x[:,0]
    T[0,0] = t
    for it in range(1,N_steps):  
            t += dt
        
            obj_w.x = x[0:1,:]
            obj_pr.x = x[1:6,:]
            
            obj_w.tau_r = obj_pr.tau_r
            obj_pr.omega_r = obj_w.x[0,0]
            
            obj_w.f_eval()
            obj_pr.f_eval(t)
            
            
            
            f[0:1] = obj_w.f
            f[1:6] = obj_pr.f
            x = x + dt*f
            
            X[:,it] = x[:,0]
            out[0,it] =obj_w.p_m
            T[it,0] = t
            
    return T,X,out,obj_w


In [309]:
V_dc_src = np.arange(20,150,10)
P_m = []
Omega = []
for v_dc_src in V_dc_src:
    
    T,X,out,obj_w = run_eval(nu=12,v_dc_src=v_dc_src)
    P_m += [obj_w.p_m]
    Omega += [X[0,-1]]
    

In [301]:
fig = figure(width=600, height=300, y_axis_label='Speed (rad/s)', x_axis_label='Time (s)')
decim = 100
fig.line(T[0:-1:decim,0],X[0,0:-1:decim], line_width=2, color='red')
show(fig)

fig = figure(width=600, height=300, y_axis_label='Power (kW)', x_axis_label='Time (s)')
decim = 100
fig.line(T[0:-1:decim,0],out[0,0:-1:decim]/1000, line_width=2, color='green')
show(fig)

In [310]:
omega_t = np.linspace(0.1,200/10,100)
NU = [4,6,8,10,12,14,obj_w.nu]
beta = 0.0

fig = figure(width=600, height=300, y_range=[-5,300], y_axis_label='Power (kW)', x_axis_label='Speed (rad/s)')
decim = 1
for nu in NU:
    fig.line(omega_t,obj_w.aero(omega_t,nu,beta)/1000, line_width=2, color='blue')
    fig.circle(X[0,-1],obj_w.aero(X[0,-1],obj_w.nu,beta)/1000, size=8, color='red')
show(fig)



In [311]:
fig = figure(width=600, height=300, y_axis_label='Speed (rad/s)', x_axis_label='V_dc')
fig.line(V_dc_src, Omega, line_width=2, color='red')
show(fig)
fig = figure(width=600, height=300, y_axis_label='Power (kW)', x_axis_label='V_dc')
fig.line(V_dc_src, np.array(P_m)/1000, line_width=2, color='red')
show(fig)

In [ ]:
fig = figure(width=600, height=300)
decim = 100
fig.line(T[0:-1:decim,0],X[0,0:-1:decim], line_width=1, color='red')
fig.line(T[0:-1:decim,0],X[1,0:-1:decim], line_width=1, color='green')    
fig.line(T[0:-1:decim,0],X[2,0:-1:decim], line_width=1, color='blue')  
show(fig)

fig = figure(width=600, height=300)
decim = 10
fig.line(T[0:-1:decim,0],out[3,0:-1:decim], line_width=1, color='red')
show(fig)